In [1]:
import data_loader
import numpy as np
import keras
import pickle
import os 

Using TensorFlow backend.


### Load the original data

In [2]:
img_dir_path = 'origin/images'
label_file_path = 'origin/data/legend.csv'
valid_rate = 0.1

train_file_paths, \
y_train, \
valid_file_paths, \
y_valid, \
test_file_paths, \
y_test, \
label_dict = data_loader.load_dataset(img_dir_path, label_file_path, valid_rate)

print('train_images size is: ', len(train_file_paths))
print('valid_images size is: ', len(valid_file_paths))
print('test_images size is: ', len(test_file_paths))
print('label_dict size is: ', len(label_dict))
print(label_dict.keys())

train_images size is:  11086
valid_images size is:  1226
test_images size is:  1378
label_dict size is:  8
dict_keys(['sadness', 'contempt', 'surprise', 'happiness', 'fear', 'disgust', 'neutral', 'anger'])


### Load the extracted features by CNN

In [3]:
with open("extract/x_train_norm", "rb") as f:
    x_train = np.array(pickle.load(f))

with open("extract/x_valid_norm", "rb") as f:
    x_valid = np.array(pickle.load(f))

with open("extract/x_test_norm", "rb") as f:
    x_test = np.array(pickle.load(f))

print("x_train is", x_train.shape)
print("x_valid is", x_valid.shape)
print("x_test is", x_test.shape)

x_train is (11086, 512)
x_valid is (1226, 512)
x_test is (1378, 512)


In [4]:
y_train.shape

(11086,)

In [4]:
print(x_train[0])

[0.04668649 0.01834312 0.01563531 0.03335703 0.00860873 0.00199967
 0.01166063 0.07229225 0.06217672 0.00775019 0.04941142 0.01490266
 0.02223296 0.10163697 0.0308977  0.01733001 0.00676396 0.
 0.01707973 0.0636246  0.03180046 0.03533023 0.06297326 0.02789397
 0.01069004 0.0029924  0.01485889 0.00663875 0.         0.0546362
 0.00112814 0.05854592 0.00764264 0.03521162 0.00270499 0.02090574
 0.04938011 0.04952344 0.04465076 0.02944676 0.01996635 0.01183683
 0.03067296 0.00522783 0.02979122 0.00141997 0.00572725 0.05193046
 0.0477648  0.01046932 0.         0.01858355 0.00073811 0.0166754
 0.0688165  0.056336   0.03168477 0.00173453 0.00362746 0.09754843
 0.03784222 0.02455144 0.02200243 0.01159612 0.12285333 0.01068822
 0.03498529 0.08806266 0.00112241 0.01026112 0.04767127 0.02294808
 0.01352609 0.05342175 0.08507624 0.01754083 0.04124191 0.02862307
 0.00799282 0.0211107  0.         0.02313879 0.06126232 0.00267217
 0.05358624 0.03952111 0.00935776 0.07391356 0.02166037 0.01442613
 0.01

In [4]:
import matplotlib.pyplot as plt
import pandas as pd

train_data = pd.read_csv('./origin/data/legend.csv')
test_data = pd.read_csv('./origin/data/500_picts_satz.csv')
print(test_data.keys())

print(train_data['emotion'][0])
print(train_data['emotion'][1])
print(train_data['emotion'][2])
print(train_data['emotion'][3])
print(train_data['emotion'][4])

dic = {'anger':0, 'contempt':1, 'disgust':2, 'fear':3,
      'happiness':4, 'neutral':5, 'sad':6, 'sadness':7,
      'surprise':8}

train_emotion = []
test_emotion = []
print(len(train_data['emotion']))
print(len(test_data['emotion']))

for i in range(len(train_data['emotion'])):
    tmp = train_data['emotion'][i].lower()
    tmp = int(dic[tmp])
    train_emotion.append(tmp)

y_train = train_emotion[0:11086]
y_valid = train_emotion[11086:11086+1226]
y_test = train_emotion[11086+1226:11086+1226+1378]

y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

print("y_train is", y_train.shape)
print("y_valid is", y_valid.shape)
print("y_test is", y_test.shape)

print('sample: ' + str(y_train[0:5]))

Index(['user.id', 'image', 'emotion'], dtype='object')
anger
surprise
disgust
fear
neutral
13690
500
y_train is (11086,)
y_valid is (1226,)
y_test is (1378,)
sample: [0 8 2 3 5]


In [10]:
train_data.head()

,user.id,image,emotion
0,628,facial-expressions_2868588k.jpg,anger
1,628,facial-expressions_2868585k.jpg,surprise
2,628,facial-expressions_2868584k.jpg,disgust
3,628,facial-expressions_2868582k.jpg,fear
4,dwdii,Aaron_Eckhart_0001.jpg,neutral


In [30]:
test_data.head()

,user.id,image,emotion
0,906,Sourav_Ganguly_0003.jpg,fear
1,906,Sourav_Ganguly_0004.jpg,happiness
2,906,Sourav_Ganguly_0005.jpg,neutral
3,906,Spencer_Abraham_0001.jpg,anger
4,906,Spencer_Abraham_0003.jpg,anger


In [13]:
train_data.shape

(13690, 3)

In [9]:
CScale = [-5, -3, -1, 1, 3, 5,7,9,11,13,15]
gammaScale = [-15,-13,-11,-9,-7,-5,-3,-1,1,3]

CScale = [2 ** i for i in CScale]
gammaScale = [2 ** i for i in gammaScale]

print(CScale)
print(gammaScale)

[0.03125, 0.125, 0.5, 2, 8, 32, 128, 512, 2048, 8192, 32768]
[3.0517578125e-05, 0.0001220703125, 0.00048828125, 0.001953125, 0.0078125, 0.03125, 0.125, 0.5, 2, 8]


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

parameters = {'kernel':['rbf'], 'C':CScale, 'gamma':gammaScale}
svr = svm.SVC(decision_function_shape='ovo')
clf = GridSearchCV(svr, parameters, n_jobs=-1)
clf.fit(x_train, y_train)
print('The parameters of the best model are: ')
print(clf.best_params_)
train_acc = clf.score(x_train, y_train)
print('train accuracy:{}'.format(train_acc))
test_acc = clf.score(x_test, y_test)
print('val accuracy:{}'.format(test_acc))

The parameters of the best model are: 
{'C': 0.03125, 'gamma': 3.0517578125e-05, 'kernel': 'rbf'}


In [6]:
minCScale = 0.5 * (0.5 + 2)
maxCScale = 0.5 * (2 + 8)
newCScale = np.linspace(minCScale, maxCScale, 10)

minGammaScale = 0.5 * (2 + 0.5)
maxGammaScale = 0.5 * (2 + 8)
newGammaScale = np.linspace(minGammaScale, maxGammaScale, 10)

print(newCScale)

[1.25       1.66666667 2.08333333 2.5        2.91666667 3.33333333
 3.75       4.16666667 4.58333333 5.        ]


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

parameters = {'kernel':['rbf'], 'C':CScale, 'gamma':gammaScale}
clf = svm.SVC(decision_function_shape='ovr')
clf = GridSearchCV(svr, parameters, n_jobs=-1)
clf.fit(x_train, y_train)
print('The parameters of the best model are: ')
print(clf.best_params_)

The parameters of the best model are: 
{'C': 0.03125, 'gamma': 3.0517578125e-05, 'kernel': 'rbf'}


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')

scores = cross_val_score(clf, x_valid, y_valid, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.50 (+/- 0.01)


In [22]:
from sklearn.metrics 
import roc_curve, auc
y_score = classifier.fit(x_train, y_train).decision_function(x_test) # 计算每一类的ROC 
fpr = dict() 
tpr = dict() 
roc_auc = dict() 
for i in range(n_classes): 
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i]) 
    roc_auc[i] = auc(fpr[i], tpr[i])

# val_res = clf.predict_proba(x_valid)
# fpr, tpr, thresholds = roc_curve(y_valid, val_res[:,1])
# plt.plot(fpr, tpr)
# plt.plot([0,1],[1,0],'--',color=(0.6,0.6,0.6),label='luck')
# plt.show()

ValueError: Data is not binary and pos_label is not specified

In [5]:
from sklearn import svm

In [27]:
# clf = svm.SVC(decision_function_shape='ovo',probability=True)
# clf.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
#set best params
params_svm = {'C': 0.03125, 'gamma': 3.0517578125e-05, 'kernel': 'rbf'}
clf = svm.SVC(decision_function_shape='ovr',probability=True)
clf.set_params(**params_svm)
clf.fit(x_train, y_train)
train_acc = clf.score(x_train, y_train)
print('train accuracy:{}'.format(train_acc))
test_acc = clf.score(x_test, y_test)
print('val accuracy:{}'.format(test_acc))

train accuracy:0.5261591196103194
val accuracy:0.30406386066763424


In [9]:
train_acc = clf.score(x_train, y_train)

In [10]:
print('train accuracy:{}'.format(train_acc))

train accuracy:0.5261591196103194


In [12]:
val_acc = clf.score(x_valid, y_valid)
# val_res = clf.predict_proba(x_valid)
print('val accuracy:{}'.format(val_acc))

val accuracy:0.5024469820554649


In [11]:
test_acc = clf.score(x_test, y_test)
print('val accuracy:{}'.format(test_acc))

val accuracy:0.30406386066763424


In [19]:
params_svm = {'C': 0.03125, 'gamma': 3.0517578125e-05, 'kernel': 'linear'}
clf = svm.SVC(decision_function_shape='ovo',probability=True)
clf.set_params(**params_svm)
clf.fit(x_train, y_train)
train_acc = clf.score(x_train, y_train)
print('train accuracy:{}'.format(train_acc))
test_acc = clf.score(x_test, y_test)
print('val accuracy:{}'.format(test_acc))

train accuracy:0.5261591196103194
val accuracy:0.30406386066763424


In [24]:
params_svm = {'C': 128, 'gamma': 3.0517578125e-05, 'kernel': 'poly'}
clf = svm.SVC(decision_function_shape='ovo',probability=True)
clf.set_params(**params_svm)
clf.fit(x_train, y_train)
train_acc = clf.score(x_train, y_train)
print('train accuracy:{}'.format(train_acc))
test_acc = clf.score(x_test, y_test)
print('val accuracy:{}'.format(test_acc))

train accuracy:0.5261591196103194
val accuracy:0.30406386066763424


In [ ]:
CScale =[0.03125, 0.125, 0.5, 2, 8, 32, 128, 512, 2048, 8192, 32768]
gammaScale =[3.0517578125e-05, 0.0001220703125, 0.00048828125, 0.001953125, 0.0078125, 0.03125, 0.125, 0.5, 2, 8]
clf = svm.SVC()
for kernel in ('linear', 'poly', 'rbf'):
    for gamma in gammaScale:
        for C in CScale:
            params_svm = {'C': C, 'gamma': gamma, 'kernel': kernel}
            clf.set_params(**params_svm)
            clf.fit(x_train, y_train)
            train_acc = clf.score(x_train, y_train)
            print('when kernel = '+str(kernel)+" gamma = "+str(gamma)+" C = "+str(C)+ ' train accuracy:{}'.format(train_acc))
            test_acc = clf.score(x_test, y_test)
            print('when kernel = '+str(kernel)+" gamma = "+str(gamma)+" C = "+str(C)+ ' val accuracy:{}'.format(test_acc))